In [1]:
from spacy import Language
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
from pathlib import Path

spacy.require_gpu()
# Load a blank English pipeline with transformer
nlp = spacy.blank("en")
print(nlp.use_gpu)
# Add transformer + NER to the pipeline
transformer_config = {
    "model": {
        "@architectures": "spacy-transformers.TransformerModel.v3",
        "name": "nlpaueb/legal-bert-base-uncased",
        "tokenizer_config": {"use_fast": True},
        "transformer_config": {"output_attentions": False},
    }
}
nlp.add_pipe("transformer", config=transformer_config)
nlp.add_pipe("ner", last=True)


In [20]:
import json
from docx import Document

def load_docx_text(docx_path):
    doc = Document(docx_path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

file_path = ["D:\\NEU\Summer2025\\NLP\FinalProject\Project 1\\Lease-Buddy\\backend\\tagged_dataset3.json" ,"D:\\NEU\Summer2025\\NLP\FinalProject\Project 1\\Lease-Buddy\\backend\\tagged_dataset.json","D:\\NEU\Summer2025\\NLP\FinalProject\Project 1\\Lease-Buddy\\backend\\tagged_dataset4.json"]
# Load your JSON object
TRAIN_DATA = []
for file in file_path:
    with open(file, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    print(file)

    for entry in raw_data:
        file_name = entry["file_path"].split("/")[-1].split(".")[0]
        doc_path = f"D:\\NEU\\Summer2025\\NLP\\FinalProject\\Lease-Buddy\\backend\\models\\NER\\Master Dataset\\{file_name}.docx"
        doc = Document(doc_path)
        text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()]).strip()
        ents = []

        for label, span in entry["entities"].items():
            start = span["start"]
            end = span["end"]
            entity_text = text[start:end]

            # Optional: basic sanity check
            if text[start:end].strip():
                ents.append((start, end, label))

        TRAIN_DATA.append((text, {"entities": ents}))


D:\NEU\Summer2025\NLP\FinalProject\Project 1\Lease-Buddy\backend\tagged_dataset3.json
D:\NEU\Summer2025\NLP\FinalProject\Project 1\Lease-Buddy\backend\tagged_dataset.json
D:\NEU\Summer2025\NLP\FinalProject\Project 1\Lease-Buddy\backend\tagged_dataset4.json


In [21]:
print(len(TRAIN_DATA))

72


In [23]:
ner = nlp.get_pipe("ner")
ner.add_label("LESSOR_NAME")
ner.add_label("LESSEE_NAME")
ner.add_label("PROPERTY_ADDRESS")
ner.add_label("LEASE_START_DATE")
ner.add_label("LEASE_END_DATE")
ner.add_label("RENT_AMOUNT")
ner.add_label("SECURITY_DEPOSIT_AMOUNT")

# Convert training data to spaCy examples
examples = []
for text, annot in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annot)
    examples.append(example)


In [24]:
from spacy.training import Example
from spacy.util import minibatch
from tqdm import tqdm

optimizer = nlp.initialize()

for i in range(100):  # epochs
    losses = {}
    batches = minibatch(examples, size=2)
    for batch in tqdm(batches):
        nlp.update(batch, losses=losses)
    print(f"Epoch {i + 1}, Losses: {losses}")


36it [01:11,  2.00s/it]


Epoch 1, Losses: {'transformer': 0.0, 'ner': np.float32(10602.237)}


36it [01:11,  1.98s/it]


Epoch 2, Losses: {'transformer': 0.0, 'ner': np.float32(1517.2496)}


36it [01:10,  1.95s/it]


...

Epoch 98, Losses: {'transformer': 0.0, 'ner': np.float32(13.290091)}


36it [05:35,  9.33s/it]


Epoch 99, Losses: {'transformer': 0.0, 'ner': np.float32(15.444285)}


36it [02:44,  4.58s/it]

Epoch 100, Losses: {'transformer': 0.0, 'ner': np.float32(11.513182)}


In [25]:
nlp.to_disk("custom_legal_ner_spacy_100")
# Later
nlp = spacy.load("custom_legal_ner_spacy_100")

In [26]:
doc = nlp("This Lease Agreement (\"Agreement\") is entered into on January 1, 2005, by and between: LESSOR: Union Pacific Railroad Company (\"Landlord\") LESSEE: CXT Incorporated (\"Tenant\")PROPERTY: The Landlord hereby leases to the Tenant the residential property located at: Grand Island, Nebraska 1. TERM OF LEASE The term of this lease shall commence on January 1, 2005 and shall terminate on December 31, 2009. This Agreement shall be considered a fixed-term lease. 2. RENT The Tenant agrees to pay the Landlord a monthly rent of $1,378. Rent is due on the 1st day of each month. If rent is not received by the 5th day of the month, a late fee of $50.00 will be assessed. 3. SECURITY DEPOSIT Upon execution of this Agreement, Tenant shall deposit with Landlord the sum of $5,000 as a security deposit. This deposit shall be held by the Landlord as security for the faithful performance by the Tenant of all terms, covenants, and conditions of this Agreement.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Union Pacific Railroad Company LESSOR_NAME
CXT Incorporated LESSEE_NAME
December 31, 2009 LEASE_END_DATE
$1,378 RENT_AMOUNT
$5,000 SECURITY_DEPOSIT_AMOUNT


In [27]:
print(doc.ents)

(Union Pacific Railroad Company, CXT Incorporated, December 31, 2009, $1,378, $5,000)


In [2]:
import spacy
# Later
nlp = spacy.load("custom_legal_ner_spacy_100")

D:\NEU\Summer2025\NLP\Assignments\.venv\Lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict

In [15]:
import os
from docx import Document
import pandas as pd

dataframe = []
def read_docx_directory(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(folder_path, filename)
            try:
                doc = Document(file_path)
                text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
                doc_res = nlp(text)
                desired_path = file_path.replace("\\", "/")
                res = {'FILE_PATH' : desired_path,}
                for ent in doc_res.ents:
                    res[ent.label_] = ent.text
                dataframe.append(res)

            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return pd.DataFrame(dataframe)


In [16]:
results = read_docx_directory("../datasets/dataset-master/testing")

In [17]:
results.to_csv("spacy_bert_result.csv")